# <center> Intersubject Generalization in Encoding Models Exploiting Deep Neural Netweorks <center/>
    

Welcome to this tutorial, demonstrating the cracks of our project decicated to developing the new methods of intesubject generalization!
We will test different intersubject generalization algorithms (IGA) in the encoding model framework.
    
    
This tutorial is a brief and shortened version of my Master's thesis aiming to give a reader the basic idea of the project. 
Many aspects of the original project will not be mentioned or explained in details here.

    
First, let us briefly introduce the problem we tackle in this project.

## Introduction

### First of all, why are intersubject generalization algorithms (IGA) useful?
The main reason for this is that *the brain responses of every subject to the presented stimuli are different due to subject-specific noise. IGA capture the underlying brain signal shared between the subjects (i.e. invariant of the subject-specific noise)*.

There are multiple fields in congitive neroscience that may benefit from identifying the brain signal shared between the subjects. Some of the applications of the IGA are listed below.

1. Transfer learning (e.g. no training session for online Brain-Computer Interface systems), (Cohen et al. 2017).

2. Higher sensitivity of the statistical analysis, (Richard et al. 2020).

3. Sceintific generality (conclusions about the underlying source), (Richard et al. 2020).


### But what is the main idea behind IGA?
The main objective of the IGA is to find a way to project the data from individual subject space into the space shared between all the subjects (figure below illustrates this concept). 

<img src="images/picture1.png" width=700 height=700 />


### Linear and non-linear IGA
Based on the manner of projecting the data IGA can be divided into the ones using linear mathematical operations for transforming the data and the ones using deep neural networks for this purpose. We will refer to these groups as *linear* and *non-linear* IGA respectively. 

Linear IGA are commonly used whereas non-linear IGA are just beginning to emerge (the first and the only [paper](https://arxiv.org/abs/1608.04846) exploiting DNN for intersubject generalization we managed to find used Convolutional Autoencoder on fMRI data).

### Idea
Inspired by [SimCLR framework](https://arxiv.org/abs/2002.05709) recently proposed in machine learning in this project we aimed to develop non-linear IGA trained with contrastive loss, which would outperform the existing linear methods. 

We assume that the reader is already familiar with the original SimCLR framework. If it is not the case, feel free to check out the paper above.
Let us say, we have a neuroimaging dataset collected while presenting multiple stimulito several subjects.
To adapt SimCLR for this dataset, we treat the brain responses of different subjects to similar stimuli as positive samples and the responses of different subjects to different stimuli as negative samples. 
Then, for each stimulus we use the *encoder DNN* and shallow fully-connected *projection head DNN* to obtain some latent representation of the brain responses to positive and negative samples. The contrastive loss is then calculated between the outputs of projection head in such a way, that the brain responses for positive samples are as similar as possible and the brain responses for negative samples are as dissimilar as possible. 

<img src="images/picture2.png" width=700 height=700>

### DNN to train with contrastive loss
We used the [Perceiver](https://arxiv.org/abs/2103.03206) as an encoder DNN. The Perceiver is an input modality independent DNN, which potentially allows us to use it with any neuroimaging data modality (EEG, fMRI, fNIRS, etc.).
We used the [implementation of the Perceiver](https://github.com/lucidrains/perceiver-pytorch) by [Phil Wang](https://github.com/lucidrains).
The figure below provides a scheme of the Perceiver architecture.

<img src="images/picture6.png" width=700 height=700>

### Baselines
As a baseline we will compare our contrastive loss-trained non-linear IGA with the state-of-the-art linear IGA and the first non-linear IGA.

#### Multiview ICA
[Multivview ICA](https://hugorichard.github.io/software/software-3/) is a recently proposed linear IGA which was shown to outperform the other frequently used algorithms. Therefore, we use it as a state-of-the-art linear IGA.

#### Convolutional Autoencoder
[Convolutional Autoencoder](https://arxiv.org/abs/1608.04846) is a simple DNN which was to the best of our knowledge the first  non-linear IGA.

## Dataset and Encoding model
In this project we used EEG dataset recorded with Rapid serial visual presentation paradigm, (Grootswagers, Robinson, and Carlson 2019). 
Images from the [THINGS dataset](https://pubmed.ncbi.nlm.nih.gov/31613926/) grouped in train and test datasets were presented to 7 participants. EEG response to each image was recorded. 
The *encoding model* was constructed as follows. 
Each image was fed to [CORnet-S](https://arxiv.org/abs/1909.06161) DNN.
The output activations of the CORnet-S (independent variable) and the EEG responses (dependent variable) to train images were used to train the linear regression. Then the output activations of the CORnet-S for the test images and trained linear regression were used to obtain the predicted EEG response. 
<img src="images/picture3.png" width=900 height=900>

The full technical description of the dataset is available in my master's thesis which will soon be made publically available.



To assess the performance of the performance of the encoding model we used *generic decoding* procedure. I will explain it on the toy example below.
<img src="images/picture4.png" width=700 height=700>
Let's say, we have computed the correlation between every predicted and real (recorded) EEG response and obtained a correlation matrix. Each row of the correlation matrix corresponds to the real image index and each column – to the index of the predicted image. Next, for each row (real image) we sort the correlation values in descending order. So, for every real image we have the row of indices of the predicted images sorted by the strength of their correlation with the real one. After this we define the top value N = 2.  We count the number of rows, where the correlation value with the index equalling to the row number is less or equal to N. Finally, we divide this value into the number of images (4) and obtain top 2 accuracy.

We ran the encoding model on the shared space response averaged over subjects and the indivudual shared space response (without averaging).

Let us now summarize the full model pipeline.
First, intersubject generalization was used to project individual subject data into the shared space.
Second, the encoding model was run to obtained the predicted test set response.
Third, the quality of the reconstruced EEG respnse was assessed via generic decding procedure.
<img src="images/picture5.png" width=900 height=900>

## Intersubject generalization algorithms
Now we come to the cracks of the project - intersubject genralization algorithms.

As you remember, we wanted to develop non-linear IGA trained with contrastive loss, which would outperform the existing linear methods.
Therefore, we compared the performance of the encoding model on the data transformed with the state-of-the art linear IGA, the Convolutional Autoencoder, and non-linear IGA trained with Contrastive loss (based on the Perceiver).
As the control for the IGA we ran encoding model on the untransformed data.

As the state-of-the art linear IGA we used recently proposed [MultiviewICA method](https://hugorichard.github.io/software/software-3/).

## Running the experiments

Here we reproduce some steps from the analysis. Not to overwhelm this tutorial with technical details, we skip many intermediate steps and leave the most essential ones. 

### Creating an environment with the required packages
First, set the *path* variable to the top directory of your project.
In all the follow up code the path corresponds to the project directory.

In [6]:
%%bash 
path=/mnt/e/Documentos/Study/Master_thesis/intersubject_generalization/demo
cd $path

In order to run the code you shall have all the packages from *requirements.txt* installed.
After this, let us create an environment with the required packages.

In [3]:
%%bash
conda create -n env1 -f requirements.txt

-bash: line 1: conda: command not found


CalledProcessError: Command 'b'conda create -n env1 -f requirements.txt\n'' returned non-zero exit status 127.

In [12]:
import sys
sys.path.append("/c/ProgramData/Miniconda3")
sys.path

['E:\\Documentos\\Study\\Master_thesis\\intersubject_generalization\\demo',
 'C:\\ProgramData\\Miniconda3\\python37.zip',
 'C:\\ProgramData\\Miniconda3\\DLLs',
 'C:\\ProgramData\\Miniconda3\\lib',
 'C:\\ProgramData\\Miniconda3',
 '',
 'C:\\Users\\Andrei\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\ProgramData\\Miniconda3\\lib\\site-packages',
 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\locket-0.2.1-py3.7.egg',
 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\Pythonwin',
 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Andrei\\.ipython',
 '/c/ProgramData/Miniconda3',
 '/c/ProgramData/Miniconda3',
 '/c/ProgramData/Miniconda3',
 '/c/ProgramData/Miniconda3',
 '/c/ProgramData/Miniconda3']

### Preparing the data

1. Creating the *.pkl* dataset.
The collected EEG is stored in the folder 

/data/EEG.

Let us pack the data from multiple subjects into a single *.pkl* file for the train and test sessions.

The file *create_dataset_matrix.py* packs the EEG data from different *.npy* files into a one *.pkl* file for train and test sets. The *-time* parameter corresponds to the EEG time window used in the analysis in samples, here we leave it 13 40 which corresponds to 60-600 ms after the stimulus presentation. 

In [13]:
%%bash
cd code/prepare_data/
eeg_dir="data/EEG/"
dataset_dir="data/datasets/"
python create_dataset_matrix.py -inp $eeg_dir -out $dataset_dir -time 13 40

SyntaxError: invalid syntax (<ipython-input-13-28aa1e02bf7d>, line 5)

2. Creating the feature matrix

Next, we have to reshape the EEG dataset of shape *subjects x images x timepoints x channels* into the featurematix of shape *subjects x images x features*. This is the requirement of the [Multiview ICA toolbox](https://github.com/hugorichard/multiviewica/blob/master/multiviewica/_multiviewica.py) (feel free to see the documentation).

The file *create_featurematrix.py* does the job.
It accepts the directory where the EEG dataset files are stored and the output directory where the feature matrices will be saved.

In [2]:
%%bash
cd code/prepare_data/
featuremat_dir="data/featurematrices/"
python create_featurematrix.py -inp $dataset_dir -out $featuremat_dir

Traceback (most recent call last):
  File "create_featurematrix.py", line 7, in <module>
    import joblib
ImportError: No module named joblib


CalledProcessError: Command 'b'cd code/prepare_data/\nfeaturemat_dir="data/featurematrices/"\npython create_featurematrix.py -inp $dataset_dir -out $featuremat_dir\n'' returned non-zero exit status 1.

3. Creating the feature matrix with increment of the training data

Linear methods are known to require less training data, than DNN-based. In orded to check how does the amount the the training data influence the performance of the IGA, we will create the feature matrices with sequentially increasing number of the train images. In order to do it we randomly select 10, 20, 40, 60, and 80 % of the training images. For reproducibility in the project we used the average performance over 10 random shuffles. However, in this tutorial we will only use 1 shuffle. 

Great! Now we have the feature matrices for the train and test set EEG responses and are ready to run intersubject generalization on it.

### Intersubject generalization
Now let us run different intersubject generalization methods and see how it influences the performance of the encoding model.

For every IGA we will:
1. Run the IGA on the EEG data;
2. Run linear regression (average and subject-wise);
3. Run generic decoding.
In the end we will plot the generic decoding as bar plots for average and subject-wise data. This will allow us to compare the performance of the encoding model on the data transformed by each of the IGA.

#### 1. Control
In order to have a control for the IGA, we first run the encoding model on the untransformed EEG data.

1. Running linear regression

In [ ]:
%%bash
cd code/linear/regression/
featuremat_dir=$path"/data/featurematrices/"
control_dir_av=$path"/data/regression/control/average"
control_dir_sw=$path"/data/regression/control/subjectwise"
dnn_dir=$path"/data/dnn_activations/"

# running linear regression on average data
python linear_regression_average.py -eeg_dir $featuremat_dir -dnn_dir $dnn_dir -out_dir $control_dir_av

# running linear regression on subjectwise data
python linear_regression_subjectwise.py -eeg_dir $featuremat_dir -dnn_dir $dnn_dir -out_dir $control_dir_sw

2. Running generic decoding

In [ ]:
%%bash
cd code/linear/generic_decoding/

ctrl_gen_dec_av
ctrl_gen_dec_sw=$path"/data/generic_decoding/control/"
control_dir_av=$path"/data/regression/control/average"
control_dir_sw=$path"/data/regression/control/subjectwise"
dnn_dir=$path"/data/dnn_activations/"

# running linear regression on average data
python linear_regression_average.py -eeg_dir $featuremat_dir -dnn_dir $dnn_dir -out_dir $control_dir_av

# running linear regression on subjectwise data
python linear_regression_subjectwise.py -eeg_dir $featuremat_dir -dnn_dir $dnn_dir -out_dir $control_dir_sw

In [ ]:
Let us run MultiviewICA on the featurematrices.
I wrapped the original code for the Multiview ICA into an object *intersubject_generalizer*. Following the scikit learn convention, it implements the *.fit* and *.project* methods.  